In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import random

In [3]:
df = pd.read_csv('SOLUSTDAtas_tratado.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36400 entries, 0 to 36399
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    36400 non-null  object 
 1   open    36400 non-null  float64
 2   high    36400 non-null  float64
 3   low     36400 non-null  float64
 4   close   36400 non-null  float64
dtypes: float64(4), object(1)
memory usage: 1.4+ MB
None


# Preprocesado de Datos

In [4]:
tamanio = df.shape[0]

In [5]:
NUMHORAS = 7

In [6]:
df_train = df.copy().loc[0:int(tamanio*0.7)]
df_vali = df.copy().loc[int(tamanio*0.7 + 1):int(tamanio*0.9)]
df_test = df.copy().loc[int(tamanio*0.9 + 1):tamanio]

# Desarrollo del modelo

In [7]:
class position:
    def __init__(self, value):
        self.value = value
    
    def getValue(self, valores : list[float]):
        return valores[self.value]
    
    def display(self):
        return ("p"+ str(self.value))

In [8]:
class constant:
    def __init__(self, value):
        self.value = value
    
    def getValue(self, valores : list[float]):
        return self.value
    
    def display(self):
        return ("c"+ str(self.value))

In [9]:
def aniadirNum(n:int):
        if random.randint(0,2) == 0:
            return constant(random.uniform(0.0, 5.0))
        else:
            return position(random.randrange(0, n))

In [10]:
class RegresionSimbolica:
    def __init__(self, funcionOptimizacion: callable, operations: list[list], maxSize: int, minSize: int, n : int): # n es la cantidad de horas anteriores que se tienen en cuenta
        self.funcionOptimizacion = funcionOptimizacion
        self.operations = operations  # La primera lista contendrá las operaciones más prioritarias y así progresivamente
        self.maxSize = maxSize
        self.minSize = minSize
        self.genes: list[list] = []
        self.n = n
        pass
    
    def create_gen(self, r : int):
        aux = []
        aux.append(aniadirNum(r))
        for i in range(random.randint(self.minSize, self.maxSize - 1)):
            numLO = random.randint(0, len(self.operations) - 1)
            aux.append(self.operations[numLO][random.randint(0, len(self.operations[numLO]) - 1)])
            aux.append(aniadirNum(r))
        return aux
    
    
    def __create_priv(self, r : int, numGenes: int):
        genes = []
        for j in range(0, numGenes):
            aux = self.create_gen(r)
            genes.append(aux)
        return genes
            
    def create(self, numGenes: int):
        if numGenes > 0:
            return self.__create_priv(4*self.n - 1, numGenes)
        else:
            raise Exception("El número de genes no puede ser menor que 1")
        
    def __aplica_operacion(self, op, i, j):
        if op == '+':
            return i + j
        elif op == '-':
            return i - j
        elif op == '*':
            return i * j
        elif op == '/':
            return i / j
        elif op == '^':
            return i ** j
        
    def __evaluate(self, valores: list[float], candidato: list):
        candidato_aux = []
        for i in range(0, len(candidato) - 1, 2):
            candidato_aux.append(candidato[i].getValue(valores))
            candidato_aux.append(candidato[i+1])
        candidato_aux.append(candidato[len(candidato) - 1].getValue(valores))
        for categoria_operaciones in self.operations:
            j_offset = 0
            for j in range(1, len(candidato_aux), 2):
                indice = j + j_offset
                op = candidato_aux[indice]
                if(op in categoria_operaciones):
                    op_result = self.__aplica_operacion( op, candidato_aux[indice-1], candidato_aux[indice+1])
                    candidato_aux[indice-1] = op_result
                    del candidato_aux[indice+1]
                    del candidato_aux[indice]
                    j_offset = j_offset - 2
        return candidato_aux[0]   
    
    
    def fitness2(self, X, y, candidato: list):
        valores_generados = []
        for elem in X:
            valores_generados.append(self.__evaluate(elem, candidato))
        return self.funcionOptimizacion(valores_generados, y)
               
   
    def display(self,  candidato: list):
        cadena = ""
        for i in range(0, len(candidato)):
            if i % 2 == 1:
                cadena = cadena + candidato[i] + ' '
            else:
                cadena = cadena + candidato[i].display() + ' '
        return cadena
        
    
    def mutate2(self, candidato: list):
        numberC = (1 + len(candidato)) / 2
        if numberC < self.maxSize and random.randint(0,50) == 0:
            op_cat = random.randint(0, len(self.operations) - 1)
            candidato.append(self.operations[op_cat][random.randint(0, len(self.operations[op_cat]) - 1)])
            candidato.append(aniadirNum(4*self.n))
            numberC += 2
        elif numberC > self.minSize and random.randint(0,50) == 0:
            indice = random.randrange(0, len(candidato) - 1)
            del candidato[indice]
            del candidato[indice]
            numberC -= 2
        return candidato
        
        
    def run2(self, numGenes, X, y, baremo: float):
        num_veces = 0
        best = 1000000
        candidato_best = []
        self.genes = self.create(numGenes)
        for i in range (0, len(self.genes)):
            value = self.fitness2(X, y, self.genes[i])
            if(value < best):
                best = value
                candidato_best = self.genes[i]
        while(best > baremo and num_veces < 100000):
            for j in range (0, len(self.genes)):
                self.genes[j] = self.mutate2(self.genes[j])
            dicc_aux = {}
            genes_aux = self.genes.copy()
            values_list = []
            for r in range (0, len(self.genes)):
                value = self.fitness2(X, y, self.genes[r])
                values_list.append(value)
                dicc_aux[value] = r
                if(value < best):
                    best = value
                    candidato_best = self.genes[r]
            values_array = np.array(values_list)
            values_array.sort()
            indice =  dicc_aux[values_array[values_array.size - 1]]
            del self.genes[indice]
            aux = self.create_gen(4*self.n - 1)
            self.genes.append(aux)
            num_veces += 1
            print(best)
        print("El mejor gen tiene un rendimiento de " + str(best))
        self.display(candidato_best)
        return (candidato_best, best)
    
    def runcopy(self, numGenes, X, y, baremo: float):
        num_veces = 0
        best = 1000000
        candidato_best = []
        self.genes = self.create(numGenes)
        for i in range (0, len(self.genes)):
            value = self.fitness2(X, y, self.genes[i])
            if(value < best):
                best = value
                candidato_best = self.genes[i]
        while(best > baremo and num_veces < 100000):
            for j in range (0, len(self.genes)):
                self.genes[j] = self.mutate2(self.genes[j])
            dicc_aux = {}
            genes_aux = self.genes.copy()
            values_list = []
            for r in range (0, len(self.genes)):
                value = self.fitness2(X, y, self.genes[r])
                values_list.append(value)
                dicc_aux[value] = r
                if(value < best):
                    best = value
                    candidato_best = self.genes[r]
            values_array = np.array(values_list)
            values_array.sort()
            if(num_veces > 200):
                aux = self.genes[dicc_aux[values_array[0]]]
                for ind in range(1,20):
                    indice =  dicc_aux[values_array[values_array.size - ind]]
                    del self.genes[indice]
                for rs in range(1,20):
                    self.genes.append(aux)
            num_veces += 1
            print(f"Vez num:{num_veces}, valor{best}")
        print("El mejor gen tiene un rendimiento de " + str(best))
        self.display(candidato_best)
        return (candidato_best, best)
        

In [11]:
def funcion_optimizacion_mape(valores_generados, y):
    suma = 0
    n = len(valores_generados)
    for i in range(0,n):
        suma = abs(valores_generados[i] - y[i])/y[i] +  suma
    error_medio = suma/n
    return error_medio*100 

In [12]:

operations = [
    ['*', '/'],   # Primera lista de operaciones prioritarias
    ['+', '-']    # Segunda lista de operaciones con menor prioridad
]

# Declarar el objeto
objeto_regresion = RegresionSimbolica(
    funcionOptimizacion=funcion_optimizacion_mape,  # Pasas tu función de optimización
    operations=operations,                          # Pasas la lista de operaciones
    maxSize=80,                                     # Tamaño máximo del cromosoma
    minSize=3,                                      # Tamaño mínimo del cromosoma
    n=NUMHORAS                                             # Cantidad de horas anteriores a considerar
)

In [13]:

candidato = [constant(3), '+', position(3), '*', constant(5)]
for i in range(0, 300):
    objeto_regresion.mutate2(candidato)
objeto_regresion.display(candidato)    

'c5 / p8 + p18 * c4.419243880450291 / c2.0143171513018236 - p26 '

Preparar X e y

In [14]:
X = []
y= []
for i in range(0, df_train.shape[0] - NUMHORAS):
    aux = []
    for r in range(0, NUMHORAS):
        aux.append(df_train.iloc[i + r].open)
        aux.append(df_train.iloc[i + r].high)
        aux.append(df_train.iloc[i + r].low)
        aux.append(df_train.iloc[i + r].close)
    X.append(aux)
    y.append(df_train.iloc[i + NUMHORAS].close)

In [ ]:
pos = objeto_regresion.runcopy(200, X, y, 0.5)

Vez num:1, valor34.587018465563084
Vez num:2, valor34.587018465563084
Vez num:3, valor23.69788330224319
Vez num:4, valor2.7336003486227485
Vez num:5, valor2.7336003486227485
Vez num:6, valor2.7336003486227485
Vez num:7, valor2.7336003486227485
Vez num:8, valor2.7336003486227485
Vez num:9, valor2.7336003486227485
Vez num:10, valor2.7336003486227485
Vez num:11, valor2.7336003486227485
Vez num:12, valor2.7336003486227485
Vez num:13, valor2.7336003486227485
Vez num:14, valor2.7336003486227485
Vez num:15, valor2.7336003486227485
Vez num:16, valor2.7336003486227485
Vez num:17, valor2.7336003486227485
Vez num:18, valor2.7336003486227485
Vez num:19, valor2.7336003486227485
Vez num:20, valor2.7336003486227485
Vez num:21, valor2.7336003486227485
Vez num:22, valor2.7336003486227485
Vez num:23, valor2.7336003486227485
Vez num:24, valor2.7336003486227485
Vez num:25, valor2.7336003486227485
Vez num:26, valor2.7336003486227485
Vez num:27, valor2.7336003486227485
Vez num:28, valor2.7336003486227485
Ve

In [11]:
pos

([27, '-', 7, '+', 7], 1.0167578391944412)

In [ ]:
with open('ecuaciones_const.txt', 'a') as archivo:
    archivo.write(str(pos[0]) + " : " + str(pos[1]) + "_nh" + str(NUMHORAS) + "\n")